损失函数之回归损失
===

# 1.均方误差-Mean Square Error
也可以叫做二次损失(Quadratic Loss)，或者L2损失(L2 Loss)。MSE是目标变量与预测值之间的距离平方和
$$MSE=\frac{\sum_{i=1}^n(y_i-y_i^p)^2}{n}$$

![images](...%0AERAREQEREBERAREQf//Z)

In [1]:
import numpy as np

def mse(true, pred):
    return np.sum((true - pred) ** 2)

# 2.平均绝对误差-Mean Absolute Error
也可以叫做L1损失(L1 Loss).MAE是目标变量和预测变量之间差异绝对值之和。因此，它在一组预测中衡量误差的平均大小，而不考虑误差的方向。(如果我们也考虑方向，那将被称为平均偏差(Mean Bias Error, MBE)，它是残差或误差之和)
$$MAE=\frac{\sum_{i=1}^n\left| y_i-y_i^p \right|}{n}$$

![images](...)

简而言之， 使用平方误差更容易求解，但使用绝对误差对离群点更加鲁棒。每当我们训练机器学习模型时，我们的目标就是找到最小化损失函数的点。当然，当预测值正好等于真实值时，这两个损失函数都达到最小值。

In [3]:
import numpy as np

def mae(true, pred):
    return np.sum(np.abs(true - pred))

# 3.MAE与MSE的比较

由于MSE对误差$\epsilon$进行平方操作($y - y_predicted = \epsilon$)，如果$\epsilon$ > 1，误差的值会增加很多。如果我们的数据中有一个离群点，$\epsilon$的值将会很高，将会远远大于$\left| \epsilon \right|$。这将使得和以MAE为损失的模型相比，以MSE为损失的模型会赋予更高的权重给离群点。以MSE为损失的模型将被调整以最小化这个离群数据点，但是却是以牺牲其他正常数据点的预测效果为代价，这最终会降低模型的整体性能。

对所有的观测数据，如果我们只给一个预测结果来最小化MSE，那么该预测值应该是所有目标值的均值。但是如果我们试图最小化MAE，那么这个预测就是所有目标值的中位数。我们知道中位数对于离群点比平均值更鲁棒，这使得MAE比MSE更加鲁棒。

使用MAE损失(特别是对于神经网络)的一个大问题是它的梯度始终是相同的，这意味着即使对于小的损失值，其梯度也是大的。这对模型的学习可不好。为了解决这个问题，我们可以使用随着接近最小值而减小的动态学习率。MSE在这种情况下的表现很好，即使采用固定的学习率也会收敛。MSE损失的梯度在损失值较高时会比较大，随着损失接近0时而下降，从而使其在训练结束时更加精确。如果离群点是会影响业务、而且是应该被检测到的异常值，那么我们应该使用MSE。另一方面，如果我们认为离群点仅仅代表数据损坏，那么我们应该选择MAE作为损失。

但是对于这两种模型来说，MAE更加关注的是大部分不离群的情况，而忽略了少部分离群的情况，因为他会尝试去接近中值；但是MSE则会关注离群的情况，因为它被离群点弄糊涂了。所以这两种情况在许多业务中都是不可取的。

| L2损失函数 | L1损失函数 |
| --------- | -------- |
| 不是非常的鲁棒 | 鲁棒 |
| 稳定解 | 不稳定解 |
| 总是一个解 | 可能多个解 |

# 4.平滑的平均绝对误差-Huber Loss
Huber Loss对数据离群点的敏感度低于平方误差损失。它在0处也可导。基本上它是绝对误差，当误差很小时，误差是二次形式的。误差何时需要变成二次形式取决于一个超参数$\delta$，该超参数可以进行微调。当$\delta \sim 0$时， Huber Loss接近MAE，当$\delta \sim \infty$（很大的数）时，Huber Loss接近MSE
$$
L_{\delta}(y, f(x))=\begin{cases}
\frac{1}{2}(y-f(x))^2 &\left| y-f(x) \right| \leq \delta \\
\delta \left| y-f(x) \right|-\frac{1}{2}\delta^2 &otherwise
\end{cases}
$$

![images](...)

使用MAE训练神经网络的一个大问题是经常会遇到很大的梯度，使用梯度下降时可能导致训练结束时错过最小值。对于MSE，梯度会随着损失接近最小值而降低，从而使其更加精确。在这种情况下，Huber Loss可能会非常有用，因为它会使最小值附近弯曲，从而降低梯度。另外它比MSE对异常值更鲁棒。因此，它结合了MSE和MAE的优良特性。但是，Huber Loss的问题是我们可能需要迭代地训练超参数$\delta$。

# 5.Log-Cosh Loss
Log-cosh是用于回归任务的另一种损失函数，它比L2更加平滑。Log-cosh是预测误差的双曲余弦的对数
$$L(y,y^p)=\sum_{i=1}^nlog[cosh(y_i^p-y_i)]$$

![images](data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJ%0AChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/%0A2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgo%0AKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCAFoAfgDASIAAhEBAxEB/8QA%0AHAABAQADAQEBAQAAAAAAAAAAAAYBBAUDAgcI/8QARxAAAQQBAgIHBgMGAwYE%0ABwAAAQACAwQFBhESIRMWMUFUldMUNVFhdbQiMnEHFUJSYoEjM5EkJUNyocFE%0AsdHwJjZTVWOFov/EABcBAQEBAQAAAAAAAAAAAAAAAAABAwL/xAAjEQEBAAIC%0AAgIDAQEBAAAAAAAAAQIREjEhIjJBAyOBE1Fh/9oADAMBAAIRAxEAPwD+qURE%0ABERAREQEREBERAREQEREBERAREQEREBERARTuA1RWyOh6WprzWY+rNUFuUPk%0A4mwtI3O7thvt8dl8xa1wMmOuXXXHww03RtmZYrywysL9gwdE9oeeLcBuzTxH%0AkN0FIi42ntSYvUJtjFTySPqPbHYjlgkhfE4tDg1zXtBB2I5bclypNTZS7mMl%0AS05hq96LGyNhtWLV4129KWteY4wI38Tg1zd9+EbnbfkdgrkU3a1lh6V+OhkJ%0A317pEQlZ0T3sruk5MbLK0GNhJ5DicN+7dbNTU+MuZqTF1X2pbUb3RPe2nMYG%0AvaN3MM3B0fENjy4t+7tQdtFEfs415R1ThMKZ5BFl7tBlt8Ta8rInHhb0gie8%0AcLw1zgCGucR2Fd92a21e3B+z/mom70/H8JAzh4dvnvvv/ZB2EXAu6uwlLM/u%0AuzcLLYfHE8iGQxxvk/Ix8gbwMc7cbNc4E7jbfcL6g1XiZ8haqQzTl1YyNlmN%0AWUV2uj3429OW9Hu3Ygji3BBHcg7qLkaf1DQz7JX432wsjDTxWKc1cPDt+FzD%0AI1vG07Hm3cdi+ZtS4qGpcsy2eGKpbbRmPRv3bM5zGtZw7bkkyM22BB4gexB2%0AUXExup8Xk8nNQpSWZZonPY6T2OYQlzHcL2tmLOjcQQQQHE7g/BbGpsp+49N5%0AbLdD0/sFSW10XFw8fAwu4d9jtvttvsUHTRQVf9oQoWrUOr6lTEshpRX+nrW3%0AWoxFJJ0YD/8ADY5p4iP4SOZO/Irq2ddYGrDDLYntxiWN03A6hY6SOJpLTJIz%0Ao+KNm4/O8BvzQVCKF/aH+0HHaawGXlqzxz5OtQfbiZ0MksIJaTH0j2DhYHkb%0ADic3i7l17Opsfjpsu6/kIRHRlhidG2CQSMdI0FrO/pXOLhsGDvA2JQUaKLs/%0AtBoQ5/DY5lHKvjyUU8gk/d1oPiMbmNDTH0XEN+M7k7cIAJ5OBW03WWKqY+Kx%0Acve0Ge1ZrxCpSnc95hlex4ETQ954OHZz9uEkcXIEBBVIpa7r3TlNlVz8g6YW%0Aqvt0PstaWwXwb7GQCNrvwjfme7v2W1Nq3DxWq1ds89mWxFHOw1Ks1hgjkJDH%0AufG0tY12x2LiAdj8EHfREQEREBERAREQEREBERAREQEREBERAREQEREBERAR%0AEQEREBERAREQEREBERAREQEREBERB+W47S2pz+z2DSeSr4WvBXqMijvV78sz%0AnSRkOYTCYGjhJaNxx9m/asaj0/kpWZDU2et4fG3oHUXwxid76jG1pnybyyOY%0A0/jMrhvw/g5H8Wx3/U0QflegKJ1Xa1XksjN/slzIQuhlxF+eKJ/R1o2HgnYY%0A3StB3HFsGktPLkF3q+HzuAzGWl0/Hi7mPydj2t8d6xJDJXm6NrHcJax/SNPA%0AHbHhO5dzO/K2RB+b5zSOcuHUFGCXGOx2oJIZblhzpGy13NiiikEcfCQ8FsQL%0Ad3t4ST2req6dy8Wtm5Cq2pjcebEk1k1r0z/bWuY4APruYI2P4i1xka4klp+P%0AK6QoIHTGkL2Jp6ChsT1XuwFB9W1wOcQ95iYzdm7eY3ae3blstzOY7Px60hze%0ACqYu5EMe6lJHcuyViCZA/iBbDJuOW3crDZfSD81uaRz1wZikf3XHjM5bgv3J%0AOmkdNXe1sLZY4xwASA9COF7iwjf8p2C2IdMZhmp7FypBjqWNkfZfPBHfnliy%0AHSNcGiSu5gjiPEQ9z2EkkEc91+hIgitB6fyuFtXfbHRVca+KOOvjob811kLm%0Al3E5r5WtLAQWjgaNhwrwyGj7tn9oMGUisVm4F74rlus4u6WS3Cx7I3AbbcOz%0Aoydz2ws+avNkCCBwunc3R1TJerx08fjyZ3vr18jNLHbc87tc6F7AyBwJ4i5n%0AESd+3dUGp8dbzmisti/9ngyF/HTV9ukLomSPjLfzcIJaC7t4Qdu7uXeRBDZb%0AQtRmjJsPp+ji8bNM+s+R0MIhY8xyse4uLW7k7NO3Lv7lqau0ZbyOpZ8xRHtX%0AtdKKlNWdl7eOaOjdIQ7ig36QHpXAtcOWw2PMr9EI3QBB+U6g0Jnf3BqHEadO%0AGZUzWOjqn2qaXeq5kAh2b+Fxe0ta3YkgtO5/F2Lq5jR+RnzmTy1OemLByVXI%0A045HO4X9FB0TmSEDdu+7tiA7bkdu5foK+dkEbdxmoLWbwGb9lxTblNlmvPV9%0Ask6MRymI8TZei3Lh0Q5FjRzI3HaeXQ0lnMNPj8jjDjbV+CXKNfBZmkjifDbu%0Ae0Nc2QMcQ9vCwEcJB/EN+QK/SEQQOltFW8JeoyPs1po4sTLTlcOJpM8k3SuL%0AW9gZuXbDfcclx49D5yph9O1Md7BVyOPxtOlNloMjPE9phADgYRHwTs5O2EhH%0A5jyBX6siAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAim9fmTq/HHH%0ANPD01+jC98Ez4n8D7cLXAPYQ4btcRyPYVnqhjvFZzzy76yCjRTnVDHeKznnl%0A31k6oY7xWc88u+sgo0U51Qx3is555d9ZOqGO8VnPPLvrIKNFOdUMd4rOeeXf%0AWTqhjvFZzzy76yCjRTnVDHeKznnl31k6oY7xWc88u+sgo0U51Qx3is555d9Z%0AOqGO8VnPPLvrIKNFOdUMd4rOeeXfWTqhjvFZzzy76yCjRTnVDHeKznnl31k6%0AoY7xWc88u+sgo0U51Qx3is555d9ZOqGO8VnPPLvrIKNFOdUMd4rOeeXfWTqh%0AjvFZzzy76yCjRTnVDHeKznnl31k6oY7xWc88u+sgo0U51Qx3is555d9ZOqGO%0A8VnPPLvrIKNFOdUMd4rOeeXfWTqhjvFZzzy76yCjRTnVDHeKznnl31k6oY7x%0AWc88u+sgo0U51Qx3is555d9ZOqGO8VnPPLvrIKNFOdUMd4rOeeXfWTqhjvFZ%0Azzy76yCjRTnVDHeKznnl31k6oY7xWc88u+sgo0U51Qx3is555d9ZOqGO8VnP%0APLvrIKNFOdUMd4rOeeXfWTqhjvFZzzy76yCjRTnVDHeKznnl31k6oY7xWc88%0Au+sgo0U51Qx3is555d9ZOqGO8VnPPLvrIKNFOdUMd4rOeeXfWTqhjvFZzzy7%0A6yCjRTnVDHeKznnl31k6oY7xWc88u+sgo0U51Qx3is555d9ZOqGO8VnPPLvr%0AIKNFOdUMd4rOeeXfWTqhjvFZzzy76yCjRTnVDHeKznnl31k6oY7xWc88u+sg%0Ao0U51Qx3is555d9ZOqGO8VnPPLvrIKNFOdUMd4rOeeXfWTqhjvFZzzy76yCj%0ARTnVDHeKznnl31l56Mh9lu6mqNntzQV8k1kQs2ZLDmNNSu8tDpHF23E5x237%0Aygp0REBERAREQTuvfcdb6rjfvYFRKd197irfVcb97AqJAXJr5qGTO2MVLG+G%0AzGwSx8e200Z5Fzf0PI/2+K6y4OqsQ7J1Y5acggylR3S1Jv5Xdhaf6XDkR8/k%0AusZLfLjO2Tcd5Fx9M5luZx5ldE6vaicYrFd/5opB2tPxHeD3jZdjdc2WXVWW%0AWbgiIjoREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQ%0AEREBERAREQEREBERAREQEWFydSZiLCY51h7HTTPcIoIGH8U0h/KxvzP/AEG5%0AVk3dRLZJuvSfL14szWxmz325o3S8LBuGMH8Tj3AnkPiV01P6UxElCCa5kXib%0AL3SJLMg7GnsEbf6WjkP7lUCZal1HOG9byFO6X996v+qs+yqqiU7pf33q/wCq%0As+yqqO1EiIgIiICIiCd197irfVcb97AqJTuvvcVb6rjfvYFRICIhQSGpas+I%0AyLdR4yNzy1vBkK7BzsQjscB3vZ2j4jcfBUtG3DeqRWasjZa8rQ9kjTycCtkj%0AcKLiB0lnBFyGAyEp6Ll+GpO7u+THnmO4Hl3rSTnNfbC/ru/paog7EWbeCIiA%0AiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiA%0AiIgIiICFEQeNqeKrWlnsSNjhjaXve47BoHaSVLaerS57IjUWQjeyINLMdXeP%0A8th7ZSO5z/8Ao3b4led5w1bl5KDCDg8fJtcd2izMNiIv+Vp2Lvidh8VYgADY%0ADYDsC0+E/wDWM/Zd/TKIizasqd0v771f9VZ9lVVEp3S/vvV/1Vn2VVFUSIiA%0AiIgKazWr6eJyppS1bk/RNgfYnhEZZXbNI6OMuBeHHic135Gu2257KlX59rDR%0A97Nah9vhjxkoMdZte5YcW2Ma6OVz3ugAYdy4Eb/iZ2DfcbAB39fe4q31XG/e%0AwKiUh+0LGwz08fce+y2WDJ48NayzIyM73YfzRhwY7t/iBVegIiIC08pRr5Kj%0ANTuRtlrzNLHtPeFuIm7PMSyWaqU0zesUbrtP5eR0lmJnHUsuO/tUI5bk9729%0Ajv7HvVUuNqfDfvepGYJTXyFZ/S1bA7Y3j4/FpHIjvBWNL5kZenIJohXv1n9D%0Aark84pB27fFp5EHvBXeXtOUZ43jeN/jtIiLhqIiICIiAiIgIiICIiAiIgIiI%0ACIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIs7IMKY1RkrM9mHBYZ5%0AbkbTeKWcc/ZYd9nPP9R5ho+PPuW/qfMswuNMrWGa3KeirQA85ZT+Vo/7nuG6%0A89K4iTGVZJrz2z5W24S25/5nbcmj4NaOQH/qu8ZqcqyztyvHH+t7DYytiMdB%0ASpM4IIWhrR3n4k/Ek8yVvLKLi3fmtJJJqMIslYQZU7pf33q/6qz7KqqJTul/%0Afer/AKqz7KqiqJERAREQEREE7rz3HX+qY772FUSnNee4q/1THfewqjQEREBE%0ARAUpqahYo3o9QYiN0lmFvDbrs7bMPwA73t7W/wBx3qrWFccuNcZ4co1Mbfr5%0AOjBcpSiWvM0PY8d4P/f5LbUW8dT8wZPy4DIS7v8A5ak7v4h8GPPb3A/DdWna%0ArljrrpPx5b8XsREXLQREQEREBERAREQEREBERAREQEREBERAREQEREBERARE%0AQEREBERAREQEREBERAXhdtQ0qs1mzI2KCJpe97uwAdpXuoyb/wCMM4YBudP4%0A+UdKewW52n8nzYwgb9xdy7l1jjvzXGeWvEe2m6k2ZyHWLKxGNxaWUK7xzgiP%0A8Z/rf3/AbD4qtTZZUyy3THHjGERFHbKIiAp3S/vvV/1Vn2VVUSndL++9X/VW%0AfZVUFEiIgIiICIiCc157ir/VMd97CqNTmvPcVf6pjvvYVRoCIiAiIgIiIPC9%0AUgvVJq1qNssErSx7HDcOB7QpnTd2XE5Hq5lJXyOawyULDzznhH8JP87e/wCI%0A2KrVxtTYZuZoCOOUwW4XdLWnb2xSDsPzHcR3jdd42dXpnnL8p27CLh6WzLsp%0AVkitxtgydR3Q24B/C/8AmH9Lu0H/ANF3FzZZdV1jlMpuCIijoREQEREBERAR%0AEQEREBERAREQEREBERAREQEREBERAREQEREBERAREQERcXU+YGIpMMMXtF+d%0A3RVa47ZJD/5AdpPcArJu6jnLKYzdc/VN6zduR6exEhjt2Gcdmw3/AMLDvsXA%0A/wA55hv+vcu/jaNfGUIKdOJsVeFoYxjewALQ0vhjiacjrMvtORtP6W3YPbI/%0A5fBoGwA7gF2tl1lZ8Z04xxvyvYiIuGjKIiKIiICndL++9X/VWfZVVRKd0v77%0A1f8AVWfZVUFEiIgIiICIofUWr3UtXtw8dqrSighgnsT2KctgO6WRzGt4mFrY%0AR/hn8byQS5uw5FB1tee4q/1THfewqiUh+0LKUIaWPoT3asV2xk8eYq75WiSQ%0AC5CSWtJ3I5HsCrx2ICIiAiIgIiIMIiyEEvqnH2YLUWdw7C+/WaRNADt7VD3s%0AP9Q7Wn48u9dnD5GtlsbBdpP44Jmh7T/2PzHYVvFRltrtJ5p1yNv+478n+0gd%0AlWZ3ZKPg1x2B+B2PxXc95r7Y2f55cp1e1oEWBsRusrhswiIgIiICIiAiIgIi%0AICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiJ2dqDVyV6vjaM1y5K2KvC0ve%0A9x2AAU/pmnYyN12oMtG6OeVvBTrv7a0J7j/W7kXf2HcVrgdb82CeeAx0v4dj%0A+G3O09vzYw/2J+ICsQNgtLOM19sJ73f0yiIs2wiIgyiIiiIiAp3S/vvV/wBV%0AZ9lVVEp3S/vvV/1Vn2VVBRIiICIiApvPaRp5m7JYmtXIG2Io4LcMDmBluJjn%0AOayTiaSAC9/5S07OIJ7FSIgnNd8sLV3/APumO+9gVGp3XnuWr9Ux33sCokBE%0ARAREQEREGEREGV4W60VuvLBYjbJDI0sexw3DgRsQV7LKJZtI6esSYLJt05kJ%0AHPjc0uxtiQ85Yx2xE/zM/wCo2+CrVydS4aLOY11Z73RTNIkgnZ+aGVvNrx8w%0Af+4WvpbMS5GGerkIxDlqThHai35E9z2/FrhzH9x3LvL2nKM8bxvG/wAd9ERZ%0AtWERFQREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQFJ6kuWMpe6v%0AYqV0cj2cd6yzn7PCe4Hue7sHwG57l0dT5k4utHFVjE+Ttu6KpBvtxv7yfg1o%0A5k9wX1pjCjDUHMfIZ7c7jNZnd+aWQ9p+Q7gO4ALvGcZyv8ZZZcrxn9dGhUgo%0AVIatSJsUETQxjG9jQOxbCIuXehERRRERBlERFEREBTul/fer/qrPsqqolO6X%0A996v+qs+yqoKJERARFwtWZHI4urVsYuhJfHTOE8MTd3lnRScPDz5byCIE89g%0ASfmA7qKf0dNmX1LkWfkhnsQzhkdiGLo2StMUbnEDc8g90jN/6FQIJ3XnuWr9%0AUx33sColO689y1fqmO+9gVEgIiICIiAiIgwiIgIiIMqZ1Ti7HTwZrDjfK0wR%0A0e+wsxb7uiP/AJt+B/VUywrjeN25yxmU052CylfM4yC9TcTFK3fY8i097SO4%0Ag7g/NdIKNyg6p5iTKxD/AHPdePb2Dsgk7BMB8DyDv7H4qxaQQCOYVzmvM6c/%0Ajyt9cu4IiLloIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgLTyt+vi8%0AfPcuyNirwtLnuP8A77T8O9bh5KNqjrfmm2388Bj5P9nb3Wph/wAT5sb2D4nc%0A9y6xx35vTPPLXidtrTGOns25c9mIizIWG8EMDv8AwsPaGf8AMe1x+PLuVQsA%0AADZZUyu7swxmM0yiIuWgiIqCIiAiIgIiICndL++9X/VWfZVVRKd0v771f9VZ%0A9lVQUSIiAo/9pcckmOxvsc1qPJi2RSZWibI+SR0EzHABz2NBEbpHBznAAtB5%0A7bGwUrrO7hXdFj83UzMxG1iOShj7spjP4gHNmrsPA/8AMOTgdj8DzDw/ZrTo%0A0aGUr42tLQjZdDX4+WJjHU3CCEcB4HOa7iAbJxNJB6T47qxUd+zd0JgzYp07%0AtWg3IbV3Xq88U87TBCXSPM46R54zI3id3NA7ArFBO689y1fqmO+9gVEp3Xnu%0AWr9Ux33sCokBERAREQEREGEREBERAREQediGOxDJDMxr4pGlj2OG4cDyII+C%0AktPSSabyw0/ceXUZd3YuZ3P8I5mBx/mb/D8W/orFc3P4iHM419WcuY7cPjlb%0AydE8c2vae4grrGzq9M/yY2+07jpLKndK5eWyJ8blNmZikQydvYJAfyys/pd/%0A0O4VEpZcbquscplNxhERR0IiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIuFq%0AnLSY6CGtQY2bK3XGKrETy325vd/S0cz/AKd6uMuV1HOWXGbrnakszZvIHTmO%0Ae5jCwPyFhp5xRH/hg/zvG/6Dc/BU9KvFUqxV68bY4YmhjGNGwaB2Ln6bw8WF%0AodC1xknkcZbEzvzTSn8zz/77AAuuusrPjOnGEvyy7ERFm0ZRERRERUEREBER%0AAREQFO6X996v+qs+yqqiU7pf33q/6qz7KqgokREBERBhfmGU1Bcf+1TDVn/v%0AmrSjtSU2QNp2Gw2QYHuMrn8HA8cfCBzPCGOdy35fqC8Jq0Ms0EssMT5YXF0b%0A3MBdGSC0lp7jsSOXcSgl/wBoVqeOpjq7MfZlhkyePLrTHRiOLa7DycC8P57f%0AwtPaFXqc16f9yVPquN+9hVGgIiICIiAiIgwiIgIiICIiAiIgnNU4qxK6HK4k%0ANGXpAmNpOwnYduKJx+B2G3wOxXSwOWr5nGxXanEGP3DmPGzmOB2c1w7iCCCF%0A0VG5qN+l8w7OVWOOLskDJxMG/AewWAPl2O27Rz7lpPecaxy/XeU6+1mi84pG%0AyxtkjcHRuAc1zTuCD3hegWbZhERAREQEREBERAREQEREBERAREQEREBEXy9w%0AY0ucdgOZJQaeZyVbEY2a7cdwwxDc7cyfgAO8k8gFyNLYyy6eTN5hnDk7TQ1s%0AR5+zQ9rYh8+8n4/otKg06szLMnK3fCUZN6Le6xKNwZj/AEjmG/3PwVkFpfSa%0AnbHH3y3evplECLNsIiICIiAiIgIiICIiAiIgKd0v771f9VZ9lVVEp3S/vvV/%0A1Vn2VVBRIiICIiAiIgnNee5Kv1XG/ewKjU5rz3JV+q4372FUaAiIgIiICIiD%0ACIiAiIgIiICIiAvmRjZWOZI0Oa4bFpG4IX0shCo3DSP0xlosHZJOKsk/u6Vx%0A36M9pgJ+XMt+Q27lZLm57FV81jZadsHgfza5p2cxw/K5p7iDzXM0rlrEjp8T%0AmBw5ekNnkdliP+GVv6947juFpfecp2xx/XeN6+lIiIs2wiIgIiICIiAiIgIi%0AICIiAiIgIiICj8/PJqDKu09SkcyrGA7Jzs/hYeyEH+Zw7fgP1XQ1TmJabYaG%0AMa2XL3CWV2O7GD+KR39LRz+fId63NOYaHCY1taFzpXkl8sz+bpZDzc9x+JK0%0Ax9ZyrHK/6XjP6361eKtBHDAxscUbQxrWjYNA5ABewRFm1kEREUREQEREBERA%0AREQEREBERAU7pf33q/6qz7KqqJTul/fer/qrPsqqCiREQEREBERBOa89yVfq%0AuN+9hVGpzXnuSr9Vxv3sKo0BERAREQEREGEREBERAREQEREBERBlT2q8PLcb%0ABkcW4R5iiS+u8/lkB/NG/wDpcOXyOxVCiuOVxu3OWMymq5Wn8vDmscyzC10b%0AgSyWF/J8Ug/Mxw7iF1FH56KTTmUfn6TXOpSANyMDG7/hHZMB8Wjt+I/RVdea%0AOxBHNC9skcjQ5rmncEHsIK6zk7nTn8eW/W9x6oiLhoIiICIiAiIgIiICIiAi%0AIgLnZ3K18NjZrtsngYOTWjdz3E7BrR3kk7bLfke2ONz3uDWNG5JOwA+akMRG%0A/VGWjzVpjhiarj+7oXj/ADHdhnI/6N37ufeuscd+b0zzy16zutzSuJnZJPmM%0AvscxdA4gDuK8Y5tib8hvzPeVTJ+iKZW5XdXDCYzUYREUdMoiIoiIgIiICIiA%0AiIgIiICIiAp3S/vvV/1Vn2VVUSndL++9X/VWfZVUFEiIgIiIC51jN4urkocd%0AayVGC/OAYq0lhrZZNzsOFhO57D2BdFQetrhu5zHYI0r7Kb5ILlq9Fj5pmHo5%0AQ6OFr2MLQ4vaC5xI4W/NwIDt689yVfqmO+9hVCpH9oTL/seOkgs1mUm5PH9N%0AC+u50jybsO3C8PAaBy7Wu/sq5AREQEREBERBhERAREQEREBERAREQEREGHsa%0A9ha8AtI2IKjcZvpLMsxUvF+5bshNF57IJDzMBPwJ3Lf7j4K0XPzWNgy2NmpW%0A27xSt2JHItPcQe4g7EFdYWTxemX5MbZvHtvopnSuSsxyyYTNPacrUaCJOz2m%0ALsbKB8e5w7j+qplMpq6d45TKbERFHQiIgIiICIiAiIgIim9V5axEYcVhyDmL%0AoPRkjcQRj80rvkN+Q7zsPirjjcrpzllxm608xIdT5d+DruIxlYj95St/jPIi%0AAH59rtu7l3qtjY2JjWMaGsaNgANgAtHA4mvhsZFSqh3Azm57ju6Rx7XOPeSe%0Aa6K6yyl8Tpzhjfle6IiLh2IiIMoiIoiIgIiICIiAiIgIiICIiAp3S/vvV/1V%0An2VVUSndL++9X/VWfZVUFEiIgIiICIiCd177jrfVMb97AqIqd177jrfVMb97%0AAqIoCIiAiIgIiIMIiICIiAiIgIiICIiAiIgIiIODqnDvyMUNmg9sOXpkyVZi%0ANxv3sd/S4cj/AK9y2NN5mPM4/pujdBZjd0Viu/8ANDIPzMP/AGPeNiuspPUd%0ASxiMiNRYqJ0jg0Mv1o+XtEQ7Hgd72d3xG4+C7xvKcb/GVnC8p/VYi1sfbgyF%0AOK1UkbLXlaHse07ggra2XFaS78sIiIoiIgIiICIF427MVSvJPZe2OGNpe97j%0AsGgcySnaW6aOosxFhca+1Kx8shIjhhZzfNIeTWN+ZP8A69y0tK4iao2fIZRz%0AZMvdIfO4dkY/hjb/AEtH+p3PetHT0E+dyI1Dko3RwtaW4yu8c44z2yuH87v+%0Ag/UquAAWl9ZxjLGc7yvX0yiIsmoiIqCIiDKIiKIiICIiAiIgIiICIiAiIgKd%0A0v771f8AVWfZVVRKd0v771f9VZ9lVQUSIiAiIgIiIJ3XvuOt9Uxv3sCoip3X%0AvuOt9Uxv3sCoigIiICIiAiIgwiIgIiICIiAiIgIiICIiAiIgIiIIt7XaOyxe%0ACRpy8/ZzR+WlM49o+Ebj/YHb4q0HZyXjbrRW60tezG2SGVpY9jhuHA9oKlML%0AZm01kIcHk3ufSlPDjrTzvv8A/hef5gOw94+fJafOb+2XwuvpYoiLNqIiICIi%0AATsFFyl2scsYmf8Ay7Rk/wARx7LkzT+X5xtI59xPLsBXtnbs+dyMmBxEjo44%0A9v3jcYf8ph/4bD/9R3/8j57Kmo04KFOGrUibFBC0MYxvIABaT0m/tjf2XX02%0AByREXDQREUUREQEREGUREUREQEREBERAREQEREBERAU7pf33q/6qz7KqqJTu%0Al/fer/qrPsqqCiREQEREBEXGyepMXjMlDQu2XR2ZeA7CGR7WB7yxhe5rS1gc%0A4FoLiNyCAg1te+4631TG/ewKiKndee4631TG/ewKiKAiIgIiICIiDCIiAiIg%0AIiICIiAiIgIiICIiAiIgLQzOLq5jHyU70YfE/v72nucD3EdoK31lJdeYlks1%0AUlhcvaxl6PBaheHWSNqdzsbbaO4/CQDbcf3Cq991oZvE1MzRdUvx8cZ5gg7O%0AY4djmntBHxCno8tkNMbQai6S3jByjyjGblg7hM0dnw4xyPfsVp4z8ztlMr+P%0Axl0sUWvTuV7sDZ6c8U8Lhu18bw4H+4WbVqCpA+azNHDEwbufI4NaP1K41WnK%0AdvdS+dzFm1ddhcAR+8SAbFkjiZTYf4j3F57m/wBzyWvNmruonGtplroaZ/DL%0AlZGbNAPaIQfzu/q/KPmu9hMPVw1T2emwgFxfJI47vkee1znd5PxXckw85ds7%0Abn4x6ZwWJr4egyrVB4Rze9x3dI8/me497ieZK6KIuLd+a0kmM1BERRRERARE%0AQEREGUREUREQEREBERAREQEREBERAU7pf33q/wCqs+yqqiU7pf33q/6qz7Kq%0AgokREBERAX59qzCZSzl8tFSpOsVszBUhNkSsaKvRSOLy8OcHEcLgW8Idu7cH%0AhHNfoKwUEl+0KpNJSx9ht+zFDHlMcHVmNj6OXe7D+Ylpdy3/AIXN7O9VxU3r%0A8SDTrJI4LE/RZChM9leF0r+BluFziGMBcdmtJ5DuWTrDG99bOj/9Hd9FBRop%0A3rjjPDZ3yO76K8W64w7rD4GxZozsY2R8Ywl3ia1xcGkjot9iWu2/Q/BBUIp3%0ArjjPC57yK76KdccZ4XPeRXfRQUSKd644zwue8iu+inXHGeFz3kV30UFCineu%0AGM8NnfIrvorysa4xFeNr54c1GwvbGHPwl0Dic4NaP8rtJIA+ZCCnRT3W/G+G%0Aznkl30k6343w2c8ku+kgoUU91vxvhs55Jd9JOt+N8NnPJLvpIKFFPdb8b4bO%0AeSXfSTrfjfDZzyS76SChRTNfW2Istc+CLMysa90ZczC3COJpLXDfou0OBB+Y%0AK9et+N8NnfI7vpIKFFPdb8b4bO+R3fSTrfjfDZ3yO76SChRT3W/G+Gzvkd30%0Ak6343w2d8ju+kgoUUxPrfEQPhZLDmmvmf0cbThbgL3cJdsB0XM8LXHl3Alev%0AXHGeGzvkd30UFEineuOM8NnfI7vop1xxnhs75Hd9FBRIQCCDzCneuOM8NnfI%0Arvop1xxnhs75Fd9FBizovBy2HzxVX1J3ncyU5nwEn4/gIG/zSDReEjmZNPVf%0AcmYd2vuTPnIPy4yQs9ccZ4bO+R3fRXjU1xh7lWKzVizM1eZgkjkZhbpa9pG4%0AcD0XMEHfdd/6Zdbcf5Yf8UzWhoAaAGjsAC+lO9cMb4XO+R3fSTrhjfC53yO7%0A6S4dqJZU71wxvhc75Hd9JOuGN8LnfI7vpIKJFO9cMb4XO+R3fSXk/W+IZYZA%0AYs1072OkZEcLcDnNaQHEDouYBc3f/mHxQU6Kc6443wme8ju+knXHG+Ez3kd3%0A0kFGinOuON8JnvI7vpJ1xxvhM95Hd9JBRopzrjjfCZ7yO76S8rOucPVryz2o%0Ac1DBEx0kkkmFutaxoG5JPRcgAgqEU71xxnhc95Fd9FOuOM8LnvIrvooKJFO9%0AccZ4XPeRXfRTrjjPC57yK76KCiRTvXHGeFz3kV30U644zwue8iu+igokUxDr%0AfETSTsjizTnwv6ORowt0ljuEO2P+FyOzmn9CF7dcMZ4bOeR3fSQUKKe64Yzw%0A2c8ju+knXDGeGznkd30kFCinuuGM8NnPI7vpJ1wxnhs55Hd9JBQopixrfEV4%0A2vnjzMbHPbGHPwtwAuc4NaP8rtJIAHxIXr1wxvhc75Hd9FBRKd0v771f9VZ9%0AlVTrhjfC53yO76K8dFTm3d1NcbXuQwWck18JtVZK7ntFSuwkNka123E1w327%0AQUFQiIgIiICIiAhCLSyuNhyldsNl9pjA4P3rWpa7t+f8Ubmnbn2b7INPUudj%0A0/WrWJ6VyzDNPHXc+uGERF72sa53E4cuJw7Nz8lyr2dxuK1ZZc+tckmkiqVb%0AdpnAYK3FJIIGv3cHbudK78rXbAtLthzXhqfTV0afGP04BYkktQWJHZXKWH7C%0AOVkmzXOEp58G2w2HPf5Hwy2mcveyVot9gjp5R9Oa9xTPc+F0LgXNj/Bs8ODW%0AtBJZtzOx32QUGo9RQ4R8EXsdy9amZJM2vUa0vEUfDxv/ABOaNm8bBsDuS4bA%0Arq0bcN6nBaqvbJBPG2WN7eYc1w3BH6ghTOosVmrOTq5TFtx/tkNe1S6GxO8R%0AmKV0ZbJxBhPEOhYeDbb8ThxcgV3dP41mGwWOxcUhkjpVo6zXuGxcGNDQT/og%0A6CIiDCk9VZeJl/HYrIYbKPr2shWjgtRPg6N0rHCZp2Mofs3oiXfg7GldfK4G%0AplJmS2pskxzG8AFbI2Kzdt+9sT2gnn2kbrRdgJDqPEWhJxY3GVpWxRyTPlld%0AO/ZvG5ztyeFgcAS4k9I75IPbO6g/deTo0IcXfyNu3FNMxlUwjhZGYw4kySMH%0AbK3s3713VHa3wVvUFBkP7lwli0GTRxWrdh4fTLiA2SIiIni2DXEBzNiAA4/m%0AVXUidDVhjkldK9jGtdI7teQOZPzKK9kREQXI1HmmYOrXldTtXZLFhlaOGt0f%0AG57t9vzua0Dl3lb2QpxX6j687p2xv23ME74X8iDyewhw7O4/EKbyulK7Me+K%0Apjo81xvaX1c5k7E0Ow32cOkEwDgT3N/vyQb9axV0/pm1ftQWKVWIT354puF8%0AkZe98rx+AkE7uOwBPd2r30/nGZj2uM07VG1UkEc1a1wcbeJoc07sc5pBB7QT%0A2EHYghcOPTN12gpNOzTwB8tWePp2vc4V3udxRxtaRu5jA7h3JB2Y3lz5dPTW%0AOvw5HK5PLtqR3LxiAhqyulZHHG3Yfic1pJLi8/lHaBz23QUCIiAiLi5DTVG/%0AbfYnsZZkj9txBlbUDOQA5MZIGjs7gO896DRuZKvb1NjKOQx2QqzRTyy0LEvR%0A9FPI2J7HbcLy4fgkeRxNbv2hbdnUHR6k/c1fGX7UzYYZ5p4jEIoGSvkY0u45%0AGuPOJ5PC1x2Hf2LkYvFZzrhYyeYgx09SIPgoPZdkc+CA7dsZi2Mjy0FzuPkA%0AAOw8XzqnTt3N26M7cZh611nQOdlBO42axZJxOZF/hAub+YAl7AeI7t23BKtk%0AREQREQTr9V025792Gvb4RYFN1zhZ0DbBjEgiJ4uLctI58PDuQN9+S1IM7jdP%0ATU8BBVvOp0hWoOtgMMNd7w1kMbyXBxc7dg3a0gcTdyN1pXNM5R2bmFf2J2Lf%0Ak2Zfd8z2zdI2INEXDwEcJkYHF+/IbgNPavrI6ZylrL2Wxmm3GX71PI2XmV/T%0ARSVzEeBjeHZzXdBGNy5u27uRRVwiIiCIiDQy9+ahDG+tjbmRke/h6KqYwWjb%0A8xMj2t2/vv8AJcG9ncZ+78PqqvXuXTPG2rUigDA9wsvi5EPc0A8TGb7kbbFb%0Amt6mcv4htTTzqsb5ZQ2y+aw6Bwh2PEI3NY/Z55DfbkCSOey1reJnn0zFi5dP%0AYGzXhbDHHj57LnVmtaBsNzCfy7N4fw8/kqqjozyWasc01aarI4c4ZiwvZ8iW%0AOc3/AEJWwuHo3F2MNp+ClbkjfKySV/DE5zmQtfI57Ymk8y1jXBgJA5NHIdg7%0AiiMFEKIMqWgyMWsKeWxj8VkYcbJ7VQfblfCI5Sx7oZAwNkL99w7Ylo7P0C7U%0A+QEU5jbFLKeMRAMaObuEuPMkDkApl+IrU9QnMYrHw1LUYfHbbG1sTbckxjP+%0AI5u5JaXE7lp5k8xz3Dow6tpyZ4Y32a41jrD6jLjmN6B87GF7ogeLi4g1rufD%0Aw/hI33GypFHRYCWHMV4/Z4n4+LIS5SOZ1wiTpZA8OBj6LbhHSu2/HvyHzC7l%0AO/ZsmoTHXiE7BKG9I57uDYH+UAdo7/8AVB1EXnPK2CCSWTfhY0uOw3OwWpDb%0AeGkWpIY5Twno2tJ4OI7NBO/M8/kg31wdW6i6t42a/Li796pBE+aeSq6EdG1o%0A35iSRpJPPYNB7P0XVdabHWsSyjboOLjA59g35fqNlyNUVBlsR0c4Ps0Fhs9i%0AIP2Mgid0jW9h5Oc1m472khBrXcnXwFiOSDGZC5cztnpPZ4HRBwkbXaDvxva0%0AAMhH8R5qlge6SJjnxuicWgljyCWnbsOxI3HyJCkdaVJstBDXlxGm8iGWXNZH%0Ak7B2b+HZpb/hktkPEeQB2Heu7io5cfUx9BxleImCHpJpDI94awcy7tJ5cyRz%0AQdVFpOtyCxLG2tK/gY1wa3hB5lw7S4DuX1BZlfO5jqs7Whzh0jg0N2Hy4t/7%0A7INtcbUOdZhH4/pqVyeK3ZjqmaAM4IHSPaxhfxOB2LngfhDj28ltyZCB8Ln1%0ALNaQscwvPStIaziHETz5ct1Pa1NrMU6dfAOxlp8NuC7KJbjmENhlZJs0MjeX%0AF3Dt3bbjt7ECfPY3OZOPDz1rrYDc4YLhDRDNZrPEro2kO4t2mJ2+7QDwOAPJ%0AbmX1dUxeWNKWrclbH0HtNmJrDFW6Z5ZHx7uDubh/CHbDm7YHdcXH4iWs6o+y%0A+FkFHK2ctX/zOkcZ2WjIxzSwEcLp37Eb7tG+wPb5ZbEW8hkMnK2CM4/IxVjd%0AD5ZYJIXxOJ/w94XdIHAMG2zSOH58qr9CRco5SbjftUfwAHhPBNufhuOi5Lco%0AzvnjeZWMY9ruEtY4u25A892jnz+CiNlERBlERAREQEREBERAREQEREGEREBE%0ARAREQEREBERAREQEREBERAUn+0DUE2AZg+iv4zHMvXzVlt5FhfFE0V5pd9uk%0AZzLomt5u7+xVi5+TxUGQu4mzM+Vr8bZdaiDCNnOMMkJDtx2cMrjy25gd24IS%0AGK/aJCcf0uVx9uB1bHMyN6yxjW14YndKGuHE4PPF0LiGhpPNu/eVs439o+Ky%0AcJGOr2bd42W1WU68teV73OjdICHslMW3Ax5O7xtw7bbkA9nIaZo5CbLvtmV4%0AylSOnYYHcI4GGQgtI5h3+K7nv3DbbbnrzaVdPXrixnctNcrT+0VrjvZxLC7g%0AcwgARBhBa5wPE1x579oGwcPD/tAjnx8NvIRSMdLFLI2COuQ5xFv2eNg3fye4%0Alg4SO078QHJdKTVN7rLjMY7T2QhbaqT2ZHSSQF0PBJG0EhspBbtJudiTzbsP%0AzbfEH7PMTFSjrGzkH9HE6NkrpW9I1xsCwJA4N/O2RoIPZy5gro9WC65RuT5n%0AJzXKscsBmcIAZo5HMc5jwIwNt427Foae3nzKK5mP17Snr46Q1bskM8dT2i4y%0AFrIq8lgN6Nr2l5cCS9nJvGG8Q3dtzSH9oeOfWnszUMnXqx1bVtkszGbTMruD%0AZeENeSCCRtxBu45gkA7fVXQGNqilFDbyDacDKjZK3Gzo7LqwaInyfg4txwM3%0A4S0HhG4I5Lch0jUrspeyW7kElOOzHDI0sLm9O8Oe7YtIJBA23G3xBVHVwWT/%0AAHtj22/ZZqrXE8LJXxPJHcQ6N72kH5OXRXG0vga+nqliCvNNO6zO6zNJKGNL%0AnuABPDG1rByaOxo+J3JJXaURhERAREQEREHKvv8A9pMbbM4LGmd3AxjjGANt%0Ahu0nc7n4nke5eFkiN1iAvMk01iGVrWtJdwjoxu7YbD8jvku02NjXOc1rQXHd%0AxA5n9V9d+6DUkrONls7rMgaw8QaWM2aO/Y8O45fNaeKqyvp1LBtSMc+GMua2%0AOMDbYHh/LvtzPJdOeCKwzgniZKzffhe0Ebr0AAAA5BB4XYzLVkYO3bcfMjmF%0ApyxOcZZIxAYpJGShzpduJ44QB2HYfhHxXTXiatczdMYIjL28fAOL/VBoWYJ+%0AhZAGsfLZl45XljixgHPuI+DRzI3XzfbM2reic10sk1dx3iicAXbcPIc+ZHDy%0A3PYV10Qc6wZ3X42kljd3Fgady4ANPPflueY/Qrz/AMdtmD/Aaxz5HEB8pdwj%0Ah57DbYdnZuumWtLg4tBc3cA7dm6FrS4OLQXDsO3MIOZOxjrVtt2c9A2BoIc7%0AhYA4v33/ALALGOghNkTQshjiYzaJrGcBJdsXOI7t+ED+xW+KtcEkQRbl3GTw%0ADm74/rzPNejoo3vY97Guew7tcRuR+iDQdIMjabHC7iqxcXSPB5PcQRwg9+wJ%0AJ/t81tCcNa5hPSTtaT0bSA523wH+n+q92taxoa0BrRyAA2CcI4uLYcQG2+3P%0A9EHFlD2WI5Z69h8z5xwtHBs4dG8cI/FyABJ57bnu58uhA50kRZPXexrQCOkL%0AT39nInmNgtlzGuc0uaCWncEjsO224/1WSAQQQNkEqwxySdJ7RjmPedyRZiLh%0Av2/8Lt5/FUGM4DUa6JoaNyCd9+LhPDuT89gtpoDQA0AAcgAsNa1jeFoAHwA2%0ACD6REQZREQEREBERAREQEREBERBhERAREQEREBERAREQEREBERAREQEREBER%0AAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQf/2Q==)

$log(cosh(x))$对于小的$x$来说，其大约等于$\frac{x^2}{2}$，而对于大的$x$来说，其大约等于$abs(x) - log(2)$。这意味着'logcosh'的作用大部分与均方误差一样，但不会受到偶尔出现的极端不正确预测的强烈影响。它具有Huber Loss的所有优点，和Huber Loss不同之处在于，其处处二次可导。为什么我们需要二阶导数？许多机器学习模型的实现（如XGBoost）使用牛顿方法来寻找最优解，这就是为什么需要二阶导数（Hessian）的原因。对于像XGBoost这样的机器学习框架，二阶可导函数更有利。

# 6.smooth L1 Loss
smooth L1说的是光滑之后的L1
$$
smooth_{L1}(y_i, \hat{y_i})=\begin{cases}
\frac{1}{2}(y_i-\hat{y_i})^2 & \left| y_i-\hat{y_i} \right| < 1 \\
\left| y_i-\hat{y_i} \right| - 0.5 & otherwise
\end{cases}
$$

![images](...)

误差在$(-1,1)$上是平方损失，其他情况是L1损失

In [4]:
import torch
loss_fn = torch.nn.SmoothL1Loss(reduce=False, size_average=False)
input = torch.autograd.Variable(torch.randn(3,4))
target = torch.autograd.Variable(torch.randn(3,4))
loss = loss_fn(input, target)
print(input); print(target); print(loss)
print(input.size(), target.size(), loss.size())

tensor([[ 1.1680, -0.7531, -0.9558, -0.6298],
        [ 0.8645, -0.6154, -0.0812, -1.1436],
        [-1.0007,  0.1825, -0.0731,  0.8301]])
tensor([[ 0.3198,  0.3127,  2.4066, -0.7760],
        [ 0.7471, -1.4110,  0.7873,  1.5462],
        [-0.4629,  0.2484, -1.3869,  0.6101]])
tensor([[3.5973e-01, 5.6576e-01, 2.8624e+00, 1.0698e-02],
        [6.8895e-03, 3.1647e-01, 3.7713e-01, 2.1898e+00],
        [1.4458e-01, 2.1701e-03, 8.1389e-01, 2.4200e-02]])
torch.Size([3, 4]) torch.Size([3, 4]) torch.Size([3, 4])
